In [1]:
# Generate stacks for picking examples of interphase and m1 cells based on LDA axis

In [2]:
!pwd
!date

/allen/aics/assay-dev/MicroscopyOtherData/Viana/projects/cvapipe_analysis/local_staging_notebooks/MovieMitosis
Thu May 19 15:44:09 PDT 2022


In [3]:
import os
import sys
import importlib
import concurrent
import numpy as np
import pandas as pd
from pathlib import Path
from tqdm.notebook import tqdm
from skimage import io as skio
import matplotlib.pyplot as plt
from aicscytoparam import cytoparam
from sklearn.decomposition import PCA
from aicsshparam import shtools, shparam
from aicsimageio import AICSImage
from aicsimageio.writers import OmeTiffWriter
from cvapipe_analysis.tools import io, viz, general, controller, shapespace, plotting

sys.path.insert(1, '../tools')
import common

In [4]:
# Controller form cvapipe_analysis
path_config = Path("/allen/aics/assay-dev/MicroscopyOtherData/Viana/projects/cvapipe_analysis/")
config = general.load_config_file(path_config)
control = controller.Controller(config)
device = io.LocalStagingIO(control)
df = device.load_step_manifest("preprocessing")
print(df.shape, control.get_staging())

(202847, 1290) /allen/aics/assay-dev/MicroscopyOtherData/Viana/projects/cvapipe_analysis/local_staging_variance


In [5]:
space = shapespace.ShapeSpace(control)
space.execute(df)

In [6]:
# local_staging_variance_edges is generated by using the output dataframe from the
# mapping process to filter out not matched cells from the full dataset.
dsname = "m1"
path_cvapipe = Path(control.get_staging()).parent
datasets = {
    dsname: {
        "control": f"{path_cvapipe}/local_staging_variance_m1m2",
        "perturbed": f"{path_cvapipe}/local_staging_m1m2"
    }}

In [7]:
smapper = shapespace.ShapeSpaceMapper(space, output_folder="./")
smapper.use_full_base_dataset()
smapper.set_make_plots_off()
smapper.set_distance_threshold(1e10)
smapper.map(datasets)
df_map = smapper.result
df_map.head()

	m1 loaded. (2201, 1255)


NUC_MEM_PC1  NUC_MEM_PC2  NUC_MEM_PC3  \
dataset structure_name CellId                                          
base    AAVS1          466245    -0.268392    -0.829342     0.047623   
                       466246    -0.243572    -0.607768     0.631536   
                       466248    -0.402700    -0.944572    -0.832112   
                       466252     0.204445     0.848568     0.504295   
                       466254    -0.560492     0.529491     0.330510   

                               NUC_MEM_PC4  NUC_MEM_PC5  NUC_MEM_PC6  \
dataset structure_name CellId                                          
base    AAVS1          466245     1.605686     2.121168     4.532400   
                       466246     0.183432    -0.023279     0.018518   
                       466248    -0.943582     1.107919     0.192331   
                       466252     1.485295     1.869869    -1.348347   
                       466254    -0.276561    -0.058453    -0.889613   

                               NUC_MEM_PC7  NUC_MEM_PC8  Dist  SelfDist  \
dataset structure_name CellId                                             
base    AAVS1          466245    -1.457945    -1.058237   NaN       NaN   
                       466246    -0.563475     0.252781   NaN       NaN   
                       466248     1.015603     2.535163   NaN       NaN   
                       466252    -0.242527     0.776478   NaN       NaN   
                       466254     1.469526    -0.531849   NaN       NaN   

                               NNCellId  Match     m1  
dataset structure_name CellId                          
base    AAVS1          466245        -1  False  False  
                       466246        -1  False  False  
                       466248        -1  False  False  
                       466252        -1  False  False  
                       466254        -1  False  False

### Control and Device for each shape matched dataset (control and perturbed)

In [8]:
importlib.reload(io)
importlib.reload(common)

<module 'common' from '/allen/aics/assay-dev/MicroscopyOtherData/Viana/projects/cvapipe_analysis/local_staging_notebooks/MovieMitosis/../tools/common.py'>

In [9]:
dsmanagers = common.setup_cvapipe_for_matched_dataset(config, datasets[dsname])

### Load representations and compute PCA and compute LDA

In [10]:
pca_lda = []
for gene in ["CTNNB1"]:#control.get_gene_names():
    
    df_gene = df_map.loc[(dsname, gene)]

    CellIds_pt = df_gene.index.values
    CellIds_ct = df_gene.NNCellId.unique()
    print(CellIds_ct[55:60], df_gene.NNCellId.mean(), df_gene.index.values.mean())
    rloader_ct = common.RepsSharedMemoryLoader(dsmanagers["control"]["control"])
    rloader_pt = common.RepsSharedMemoryLoader(dsmanagers["perturbed"]["control"])
    reps_ct = rloader_ct.load(CellIds_ct).astype(np.uint8)
    reps_pt = rloader_pt.load(CellIds_pt).astype(np.uint8)

    reps = np.concatenate([reps_ct, reps_pt], axis=0)
    print(reps_ct.shape)
    print(CellIds_ct[58], reps_ct[58].mean())
    print(f"Nct: {len(CellIds_ct)}, <rep_ct>: {reps_ct[:58].mean()}")
    print(f"Npt: {len(CellIds_pt)}, <rep_pt>: {reps_pt.mean()}")
    vsize = int(sys.getsizeof(reps)) / float(1 << 20)
    print(f"{gene}: Data shape: {reps.shape} ({reps.dtype}, {vsize:.1f}Mb)")

    npcs = np.min([32, reps.shape[0]-1])
    pca = PCA(npcs, svd_solver="full")
    pca = pca.fit(reps)
    axes = pca.transform(reps)
    axes = pd.DataFrame(axes, columns=[f"PC{i}" for i in range(1, 1+npcs)])

    groups = np.array([0]*len(CellIds_ct) + [1]*len(CellIds_pt))
    stds = axes.std(axis=0)
    axes /= stds
    axes, pca = common.sort_pcs(axes, groups, pca)
    axes["Dataset"] = groups
    axes["CellId"] = CellIds_ct.tolist() + CellIds_pt.tolist()
    axes = axes.set_index(["Dataset", "CellId"])

    lda = common.SimpleBinaryLDA()
    lda = lda.sfit(axes.values, groups)
    lda_values = lda.transform(axes.values).flatten()
    axes["LDA"] = lda_values

    pca_lda.append(axes)


[845122 946078 748281 793305 807624] 802392.7816091954 833636.908045977


  0%|          | 0/78 [00:00<?, ?it/s]

  0%|          | 0/87 [00:00<?, ?it/s]

(78, 532610)
793305 0.022575618182159553
Nct: 78, <rep_ct>: 0.02068039692626228
Npt: 87, <rep_pt>: 0.02090388969350026
CTNNB1: Data shape: (165, 532610) (uint8, 83.8Mb)


In [ ]:
OLD [845122 946078 748281 793305 807624] 802392.7816091954 833636.908045977
OLD [845122 946078 748281 793305 807624] 802392.7816091954 833636.908045977
OLD [845122 946078 748281 855926 807624] 803112.5632183908 833636.908045977
NEW [845122 946078 748281 793305 807624] 802392.7816091954 833636.908045977 

In [ ]:
dsmanagers["control"]["device"].read_parameterized_intensity_of_alias(855926, "STR").mean()/255

In [ ]:
1st 50: 0.0198489326148589 (similar)
1st 55: 0.0349524041981938 (similar)
    
Not local
Nct: 78, <rep_ct>: 0.02060200396788144
Npt: 87, <rep_pt>: 0.02090388969350026
CTNNB1: Data shape: (165, 532610) (uint8, 83.8Mb)
        
Local
Nct: 78, <rep_ct>: 0.02060200396788144
Npt: 87, <rep_pt>: 0.02090388969350026
CTNNB1: Data shape: (165, 532610) (uint8, 83.8Mb)

In [ ]:
common.run_lda_analysis(control=control, df_map=df_map, managers=dsmanagers)

### Finding cells nearest the mean of the two populations

In [ ]:
CellIds = {}
for axes, gene in zip(pca_lda, ["CTNNB1"]):#control.get_gene_names()):
    idxs = []
    for ds, df_group in axes.groupby(level="Dataset"):
        dist = np.abs(df_group.LDA.values-df_group.LDA.mean())
        nmin = np.min([15, len(dist)])
        idxs += df_group.index[np.argsort(dist)][:nmin].tolist()
    CellIds[gene] = idxs

In [ ]:
CellIds

(0, 750552),
 (0, 946211),
 (0, 943693),
 (0, 740834),

### Load single cell data and generates stacks for selected cells

In [ ]:
scale = 4
ncells = 15
box_size = 400
# z/y chop factors
yyi, yyf, yzi, yzf = 90, box_size-90, 50, box_size-50
# cell and nuc masks
yx = (300-(yyf-yyi)+300-(yzf-yzi))/300
genes = control.get_gene_names()
mode = {
    "nuc":"max",
    "mem":"max",
    "gfp":"mean"
}
args = {"gridspec_kw": {"hspace": 0}, "sharex": True, "facecolor": "white", "dpi": 150}
seg_contrast = {
    "DSP": {"vmin": 20, "vmax": 100},
    "CETN2": {"vmin": 20, "vmax": 100},
    "PXN": {"vmin": 20, "vmax": 100},
    "RAB5A": {"vmin": 20, "vmax": 100},
    "SLC25A17": {"vmin": 20, "vmax": 100},
    "SMC1A": {"vmin": 20, "vmax": 100}
}

def get_stack_with_single_cell_views(data):
    views = []
    for ncell, celldata in enumerate(data):
        fig, axs = plt.subplots(1, 3, figsize=(1*scale, 3*scale), **args)
        for ch, ax in enumerate(axs):
            ax.axis("off")
            too_big = False
            ch_used = ch+2 if ch < 2 else 3
            proj = common.Projector(celldata["img"][[0,1,ch_used]], mask_on=True, force_fit=True, box_size=box_size)
            if ch==0: #raw data
                cmap = "gray"
                mode = {"nuc":"max","mem":"max","gfp":"max"}
                proj.set_gfp_percentiles((20, 95), local=True)
            if ch==1: #seg data
                cmap = "binary"
                mode = {"nuc":"max","mem":"max","gfp":"mean"}
                vmin = 20 if gene not in seg_contrast else seg_contrast[gene]["vmin"]
                vmax = 98 if gene not in seg_contrast else seg_contrast[gene]["vmax"]
                proj.set_gfp_percentiles((vmin, vmax), local=True)
            if ch==2: #seg data
                cmap = "binary"
                mode = {"nuc":"center_nuc","mem":"center_nuc","gfp":"center_nuc"}
                proj.set_gfp_percentiles((20, 100), local=True)
            try:
                proj.set_projection_mode(ax="z", mode=mode)
                proj.compute()
                contourz = proj.get_proj_contours()
                pz = proj.projs["gfp"].copy()
                proj.set_projection_mode(ax="y", mode=mode)
                proj.compute()
                contoury = proj.get_proj_contours()
                py = proj.projs["gfp"].copy()
                im = np.concatenate([py[yyi:yyf, :], pz[yzi:yzf, :]], axis=0)
                ax.imshow(im, cmap=cmap, origin="lower", vmin=proj.gfp_vmin, vmax=proj.gfp_vmax)
                for alias_cont, alias_color in zip(["nuc", "mem"], ["cyan", "magenta"]):
                    [ax.plot(c[:,1], c[:,0]-yyi, lw=0.5, color=alias_color) for c in contoury[alias_cont]]
                    [ax.plot(c[:,1], c[:,0]+(yyf-yyi)-yzi, lw=0.5, color=alias_color) for c in contourz[alias_cont]]
            except:
                too_big = True
                print("Cell is too big")
                pass
            ax.set_ylim(1, im.shape[0])
            ax.set_xlim(1, im.shape[1])
        axs[0].set_title(f"{gene}-{celldata['Dataset']} - {(ncell+1)%(ncells+1):02d}", fontsize=10)
        axs[1].set_title(f"lda={celldata['lda']:.3f}σ", fontsize=10)
        axs[2].set_title(f"ID={celldata['CellId']}", fontsize=10)
        plt.tight_layout()
        fig.canvas.draw()
        plt.show(close=True)
        if not too_big:
            views.append(np.array(fig.canvas.renderer._renderer))
    return common.resize_and_group_images(views)

In [ ]:
df_full = device.load_step_manifest("loaddata")
df_pt = dsmanagers["perturbed"]["device"].load_step_manifest("loaddata")

In [ ]:
for gene, axes in zip(control.get_gene_names(), pca_lda):
    cellinfo = []
    for ds, CellId in tqdm(CellIds[gene], leave=False, desc=gene):
        if ds == 0:
            row = df_full.loc[CellId]
            row = common.redirect(row)
        else:
            row = df_pt.loc[CellId]
        # Have to load its own control bc channel names are different compared to main dataset
        dscontrol = dsmanagers["control" if not ds else "perturbed"]["control"]
        producer = io.DataProducer(dscontrol)
        producer.set_row(row)
        producer.load_single_cell_data()
        producer.align_data()
        data = producer.data_aligned[[3, 5, 2, 7]]
        cellinfo.append({
            "CellId": CellId,
            "Dataset": "Control" if not ds else "Perturbed",
            "lda": axes.at[(ds, CellId), "LDA"],
            "img": data
        })
    stack = get_stack_with_single_cell_views(cellinfo)
    skio.imsave(f"SingleCellsLDA-M1-{gene}.tif", stack)

In [ ]:
common.now("complete")